## Time series extraction for Landsat

plaform = google colab pulling data from google earth engine (GEE)

similar to nb - https://github.com/krishnakafle/Blog_kaflekrishna/blob/main/GEE/time-series-Savitzky-Golay-filter/point_extraction.ipynb


In [1]:
from google.colab import auth
auth.authenticate_user()

import ee
import geemap
ee.Initialize(project='ee-chris-code') # need to replace with your own project on GEE

import pandas as pd
import geopandas as gpd

In [2]:
data = gpd.read_file('./drive/MyDrive/code_google_collab/Rovai2021_supp.shp') # rename as required

# view data and make sure look as expected - currently only performing analysis on subset
data = data.head(20)
data

,Source,Study_ID,Dom_gen,Number_of_,Publicatio,ISO3,Country_Te,Longitude_,Latitude_d,Biogeograp,Coastal_ty,Stand_basa,Mean_diame,Mean_canop,Stand_dens,Abovegroun,Notes,geometry
0,Abohassan et al. 2012,530_1,Avicennia,1,Published,SAU,Saudi Arabia,39.474424,20.766031,Indo West Pacific,VI Arheic,22.77,16.7,3,1040,18.58,None,POINT (39.47442 20.76603)
1,Abohassan et al. 2012,530_2,Avicennia,1,Published,SAU,Saudi Arabia,38.195640,23.979278,Indo West Pacific,VI Arheic,9.08,9.3,2.57,1337,10.77,None,POINT (38.19564 23.97928)
2,Abreu et al. 2006,60,Rhizophora,3,Published,BRA,Brazil,-46.672167,-0.927750,Atlantic East Pacific,III Tidal system,9.04,16.6,10.63,418,51.81,None,POINT (-46.67217 -0.92775)
3,Abreu et al. 2006,1001,Rhizophora,3,Published,BRA,Brazil,-46.672167,-0.927750,Atlantic East Pacific,III Tidal system,9.11,13,8.9,418,43.7,None,POINT (-46.67217 -0.92775)
4,Accad et al. 2019,3048_1,Avicennia,NA,Published,AUS,Australia,140.846897,-17.437662,Indo West Pacific,III Tidal system,0.81,9.29,5.33,120,2.34,None,POINT (140.84690 -17.43766)
5,Accad et al. 2019,3048_10,Avicennia,NA,Published,AUS,Australia,141.668207,-15.010453,Indo West Pacific,III Tidal system,17.7,13.43,6.17,1250,58.86,None,POINT (141.66821 -15.01045)
6,Accad et al. 2019,3048_11,Avicennia,NA,Published,AUS,Australia,141.668207,-15.010453,Indo West Pacific,III Tidal system,8.65,14.03,5.32,560,24.81,None,POINT (141.66821 -15.01045)
7,Accad et al. 2019,3048_12,Avicennia,NA,Published,AUS,Australia,141.668207,-15.010453,Indo West Pacific,III Tidal system,9.75,10.53,8.14,1120,42.77,None,POINT (141.66821 -15.01045)
8,Accad et al. 2019,3048_13,Avicennia,NA,Published,AUS,Australia,141.655333,-14.974882,Indo West Pacific,III Tidal system,12.99,7.8,7.32,2720,51.25,None,POINT (141.65533 -14.97488)
9,Accad et al. 2019,3048_14,Avicennia,NA,Published,AUS,Australia,141.648818,-14.968799,Indo West Pacific,III Tidal system,15.35,7.27,8.17,3700,67.6,None,POINT (141.64882 -14.96880)


In [3]:
# Create a geemap Map
m = geemap.Map()
# Iterate through each point in the GeoDataFrame
for index, row in data.iterrows():
    # Add marker to the map
    m.add_circle_markers_from_xy(
        data, x='Longitude_', y='Latitude_d', radius=1, color="blue", fill_color="blue"
    )
# Display the map
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [4]:
# make a feature collection for GEE to be able to load EO data for
features = []
for index, row in data.iterrows():
    # construct the geometry from dataframe
    poi_geometry = ee.Geometry.Point([row['geometry'].x, row['geometry'].y])
    # construct the attributes (properties) for each point
    poi_properties = {'Study_ID':row['Study_ID']}
    # construct feature combining geometry and properties
    poi_feature = ee.Feature(poi_geometry, poi_properties)
    # append to list
    features.append(poi_feature)

# final Feature collection assembly
ee_fc = ee.FeatureCollection(features)
# ee_fc.getInfo()

In [5]:
# TODO: could get the year each sample was collected and use the appropriate landsat sensor to match dates
# for now, just using nominal year (2020) for lansat 8 for all records
# band info = https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#bands
dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate('2020-01-01', '2020-12-31').filterBounds(ee_fc).filterMetadata('CLOUD_COVER', 'less_than', 20).select('SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7','ST_B10')

# Function to reproject each image to EPSG:4326
def reproject_to_epsg4326(image):
    return image.reproject(crs='EPSG:4326', scale=30)

# Apply the reproject function to each image in the collection
reprojected_dataset = dataset.map(reproject_to_epsg4326)

# Applies scaling factors to collection 2 surface reflectance product
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

dataset = dataset.map(apply_scale_factors).map(reproject_to_epsg4326)

In [6]:
# Function to extract data from EO by feature collection (i.e. points)
def rasterExtraction(image):
    feature = image.sampleRegions(
        collection = ee_fc, # feature collection
        scale = 30 # Cell size of raster
    )
    return feature

In [7]:
# Function to manage the date formating as per requirements for output csv
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [8]:
# apply functions to dataset
dataset_extracted = dataset.filterBounds(ee_fc).map(addDate).map(rasterExtraction).flatten()


In [9]:
# check outputs are sensible and get dict to make columns for df
sample_result = dataset_extracted.first().getInfo()
sample_result

{'type': 'Feature',
 'geometry': None,
 'id': 'LC08_098070_20200414_5_0',
 'properties': {'SR_B1': 0.05245,
  'SR_B2': 0.06955499999999998,
  'SR_B3': 0.10651500000000003,
  'SR_B4': 0.121805,
  'SR_B5': 0.2819375,
  'SR_B6': 0.2594425,
  'SR_B7': 0.15604250000000003,
  'ST_B10': 315.03373952,
  'Study_ID': '3048_10',
  'date': 20200414}}

In [10]:
# extract the properties column from feature collection
# column order may not be as our sample data order
column_df  = list(sample_result['properties'].keys())
print(column_df)

['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', 'Study_ID', 'date']


In [11]:
# extract out all data
nested_list = dataset_extracted.reduceColumns(ee.Reducer.toList(len(column_df)), column_df).values().get(0)
data_output = nested_list.getInfo()

In [12]:
# dont forget we need to call the callback method "getInfo" to retrieve the data
df = pd.DataFrame(data_output, columns=column_df)

df['date']= pd.to_datetime(df['date'], format='%Y%m%d')
# we obtain the data frame as per our demand

df.to_csv('./drive/MyDrive/code_google_collab/Rovai2021_supp_landsat8_2020.csv')